In [1]:
import os

from dotenv import load_dotenv

# Explicitly providing path to '.env'
from pathlib import Path  # Python 3.6+ only
# Load .env variables
_ = load_dotenv(dotenv_path=f"{Path().resolve().parents[1]}/standalone/.env")

import import_ipynb
from utils import to_timestamp, df_data_sensors, to_be_checked, is_corr_ok, df_to_csv, df_from_csv

importing Jupyter notebook from utils.ipynb
importing Jupyter notebook from aqi_epa_pm25.ipynb
PM2.5: 35.9, AQI: 102
PM2.5: 35.9, Measure Level: MeasureLevels.UNHEALTHY_FOR_SENSITIVE_GROUPS, Range Values: Min: 35.5, Max: 55.4
AQI: 102, Measure Level: MeasureLevels.UNHEALTHY_FOR_SENSITIVE_GROUPS, Range Values: Min: 101, Max: 150


## PM2.5 Raw Data

In [2]:
# Load Tangaras
df_tangaras = df_from_csv("tangaras.csv", dtindex=False)

print(f"Total Tangara Sensors: {len(df_tangaras)}")

df_tangaras.head()

Total Tangara Sensors: 13


,ID,GEOHASH,MAC,GEOLOCATION,LATITUDE,LONGITUDE
0,TANGARA_2BBA,d29e6b4,D29ESP32DE02BBA,3.3844757080078125 -76.51634216308594,3.384476,-76.516342
1,TANGARA_260A,d29edyj,D29ESP32DE1260A,3.4613800048828125 -76.51222229003906,3.461380,-76.512222
2,TANGARA_4B1A,d29esj8,D29ESP32DE94B1A,3.4586334228515625 -76.46415710449219,3.458633,-76.464157
3,TANGARA_1EE6,d29edh3,D29ESP32DED1EE6,3.4517669677734375 -76.55067443847656,3.451767,-76.550674
4,TANGARA_2B42,d29e6pg,D29ESP32DED2B42,3.4270477294921875 -76.54792785644531,3.427048,-76.547928


In [3]:
# Start Date Time ISO 8601 Format, TZ='America/Bogota' -05:00
START_ISO8601_DATETIME=os.getenv("START_ISO8601_DATETIME", None)
start_timestamp = to_timestamp(START_ISO8601_DATETIME)
# End Date Time ISO 8601 Format, TZ='America/Bogota' -05:00
END_ISO8601_DATETIME=os.getenv("END_ISO8601_DATETIME", None)
end_timestamp = to_timestamp(os.getenv("END_ISO8601_DATETIME", None))

print(f'Since: {START_ISO8601_DATETIME} -> {start_timestamp}, Until: {END_ISO8601_DATETIME} -> {end_timestamp}')

Since: 2023-10-05T19:17:08-05:00 -> 1696551428000, Until: 2023-10-05T20:17:08-05:00 -> 1696555028000


In [4]:
# PM2.5 Data Frame Sensors
df_pm25_raw = df_data_sensors(df_tangaras, start_timestamp, end_timestamp, 'pm25')
df_pm25_raw.head()
#df_pm25_raw.shape

,TANGARA_2BBA,TANGARA_260A,TANGARA_4B1A,TANGARA_1EE6,TANGARA_2B42,TANGARA_2E9A,TANGARA_2FF6,TANGARA_307A,TANGARA_1712,TANGARA_48C6,TANGARA_532E,TANGARA_F1AE,TANGARA_06BE
DATETIME,,,,,,,,,,,,,
2023-10-05 19:17:00-05:00,3.0,4.0,49.0,NaN,NaN,16.0,3.0,9.0,14.0,NaN,23.0,NaN,NaN
2023-10-05 19:17:30-05:00,3.0,3.0,40.0,15.0,2.0,15.0,3.0,8.0,13.0,5.0,NaN,3.0,0.0
2023-10-05 19:18:00-05:00,3.0,4.0,35.0,15.0,2.0,15.0,3.0,8.0,12.0,4.0,22.0,3.0,0.0
2023-10-05 19:18:30-05:00,2.0,4.0,29.0,16.0,2.0,14.0,3.0,9.0,12.0,5.0,27.0,3.0,0.0
2023-10-05 19:19:00-05:00,3.0,4.0,28.0,15.0,2.0,15.0,3.0,9.0,12.0,5.0,23.0,3.0,0.0


## Descriptive Statistics

In [5]:
# Describe Data
df_pm25_raw.describe()

,TANGARA_2BBA,TANGARA_260A,TANGARA_4B1A,TANGARA_1EE6,TANGARA_2B42,TANGARA_2E9A,TANGARA_2FF6,TANGARA_307A,TANGARA_1712,TANGARA_48C6,TANGARA_532E,TANGARA_F1AE,TANGARA_06BE
count,111.000000,108.000000,107.000000,107.000000,109.000000,107.000000,108.000000,106.000000,108.000000,103.000000,107.000000,110.000000,107.0
mean,4.648649,4.888889,15.112150,16.700935,5.486239,20.130841,5.444444,11.745283,16.916667,6.242718,13.981308,2.872727,0.0
std,2.061106,3.867751,9.590018,8.803103,2.651396,7.842896,2.923745,1.831270,5.995520,2.064955,5.268241,0.665108,0.0
min,2.000000,2.000000,5.000000,7.000000,2.000000,11.000000,3.000000,8.000000,11.000000,2.000000,6.000000,2.000000,0.0
25%,3.000000,3.000000,8.000000,11.000000,4.000000,15.000000,4.000000,11.000000,13.000000,5.000000,10.000000,2.250000,0.0
50%,4.000000,3.000000,12.000000,13.000000,5.000000,17.000000,4.000000,12.000000,15.000000,6.000000,13.000000,3.000000,0.0
75%,6.000000,4.000000,19.500000,21.000000,7.000000,24.000000,6.000000,13.000000,18.000000,7.500000,17.000000,3.000000,0.0
max,9.000000,17.000000,49.000000,40.000000,13.000000,41.000000,12.000000,17.000000,33.000000,12.000000,27.000000,6.000000,0.0


## Missing Data

In [6]:
# Missing Data Threshold
threshold = 90
print(f'Threshold: {threshold}%')

# For each Tangara sensor
for id_tangara_sensor in df_pm25_raw.columns:
    df_pm25_sensor = df_pm25_raw[[id_tangara_sensor]]
    # To be checked
    is_ok, data_percent, missing_data_percent = to_be_checked(df_pm25_sensor, threshold)
    if not is_ok:
        print(f"Tangara Sensor: {id_tangara_sensor}, Data: {data_percent}%, Missing: {missing_data_percent}%, To be checked")

Threshold: 90%
Tangara Sensor: TANGARA_260A, Data: 89%, Missing: 11%, To be checked
Tangara Sensor: TANGARA_4B1A, Data: 88%, Missing: 12%, To be checked
Tangara Sensor: TANGARA_1EE6, Data: 88%, Missing: 12%, To be checked
Tangara Sensor: TANGARA_2E9A, Data: 88%, Missing: 12%, To be checked
Tangara Sensor: TANGARA_2FF6, Data: 89%, Missing: 11%, To be checked
Tangara Sensor: TANGARA_307A, Data: 88%, Missing: 12%, To be checked
Tangara Sensor: TANGARA_1712, Data: 89%, Missing: 11%, To be checked
Tangara Sensor: TANGARA_48C6, Data: 85%, Missing: 15%, To be checked
Tangara Sensor: TANGARA_532E, Data: 88%, Missing: 12%, To be checked
Tangara Sensor: TANGARA_06BE, Data: 88%, Missing: 12%, To be checked


## Data Correlation

In [7]:
# Data Correlation Threshold
threshold = 0.9
print(f'Threshold: {threshold}%')

ID_REFE_TANGARA_SENSOR='TANGARA_06BE'
ID_TARG_TANGARA_SENSOR='TANGARA_2FF6'
# Reference Tangara Sensor
df_reference_sensor = df_pm25_raw[ID_REFE_TANGARA_SENSOR]
# Target Tangara Sensor
df_target_sensor = df_pm25_raw[ID_TARG_TANGARA_SENSOR]

# To be checked
is_ok, corr = is_corr_ok(df_reference_sensor, df_target_sensor, threshold)
if not is_ok:
    print(f"Reference Tangara Sensor: {ID_REFE_TANGARA_SENSOR}, Target Tangara Sensor: {ID_TARG_TANGARA_SENSOR}, Correlation: {corr}, To be checked")

# For each Tangara sensor
# for id_tangara_sensor in df_pm25_raw.columns:
#     # Target Tangara Sensor
#     df_target_sensor = df_pm25_raw[id_tangara_sensor]
#     # To be checked
#     is_ok, corr = is_corr_ok(df_reference_sensor, df_target_sensor, threshold)
#     if not is_ok:
#         print(f"Reference Tangara Sensor: {ID_REFE_TANGARA_SENSOR}, Target Tangara Sensor: {id_tangara_sensor}, Correlation: {corr}, To be checked")


Threshold: 0.9%
Reference Tangara Sensor: TANGARA_06BE, Target Tangara Sensor: TANGARA_2FF6, Correlation: 0.0, To be checked


In [8]:
# Save PM2.5 Data Frame Sensors into CSV file
df_to_csv(df_pm25_raw, "pm25_raw.csv")